In [1]:
#!pip install dearpygui

In [2]:
from src.utils import *
from src.database_torrentitem import *
import dearpygui.dearpygui as dpg

In [19]:
categories = os.listdir(FOLDER_DATABASE)

In [18]:
path

'db\\anime\\56c1b17ce1cf6851038b4621.json'

In [16]:
item

56c1b179e1cf6851038b45b8: 
Fullmetal Alchemist: Premium Collection / Стальной алхимик OVA (4 из 4) Complete

['\n', '<u>Информация об аниме</u>', '<br/>', '\n', '<strong>Страна: </strong>', 'Япония', '<br/>', '\n', '<strong>Жанр: </strong>', 'приключения, фэнтези', '<br/>', '\n', '<strong>Тип: </strong>', 'OAV', '<br/>', '\n', '<strong>Год выхода: </strong>', '2006', '<br/>', '\n', '<strong>Кол серий: </strong>', '4 эп', '<br/>', '\n', '<strong>Режиссер: </strong>', 'Мидзусима Сэйдзи', '<br/>', '\n', '<strong>Продолжительность: </strong>', '10 мин', '<br/>', '\n', '<strong>Описание: </strong>', 'Все-таки японцы молодцы! ОВАшки получились своеобразным десертом к сериалу и мувику!', '<br/>', '\r\nТут для желающих отклик прошлых битв с гомункулами, не лишенный однако доли пафоса, уж слишком картинно все там появляются! Однако, сдобренный доброй долей оригинального Алхимического юмора, этот эпизод стоит того, чтобы его посмотреть и ещё раз вспомнить сериал!', '<br/>', '\r\nТут всем желающи

In [21]:
import concurrent.futures

items = {}
for category in categories:
    items[category] = []
    paths = get_json_files(os.path.join(FOLDER_DATABASE, category))
    for path in paths:
        item = TorrentItem.from_json(path)
        
        #item._local_image = change_extension(path, get_extension(item.ref_image))
        #items[category].append(item)
    #items[category].sort(key=lambda x: str(x.update_date).lower(), reverse=True)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 55: character maps to <undefined>

In [7]:
items['anime'][0].update_date

'\n\nCоздан:\xa04 января 2008 в\xa017:31\n\n'

In [12]:
len(items)

2

In [8]:
items = {}
for category in categories:
    items[category] = []
    paths = get_json_files(os.path.join(FOLDER_DATABASE, category))
    for path in paths:
        item = TorrentItem.from_json(path)
        item._local_image = change_extension(path, get_extension(item.ref_image))
        items[category].append(item)
    items[category].sort(key=lambda x: x.update_date, reverse=True)

In [3]:
all_files = get_json_files(FOLDER_DATABASE)

In [4]:
files = all_files[0:50]

In [5]:
imagePaths = []
for path in files:
    item = TorrentItem.from_json(path)
    imagePaths.append([change_extension(path, get_extension(item.ref_image)), item.guid])
    items.append(item)

In [6]:
dpg.create_context()

dpg.create_viewport(title='AnimeLayersCollection', width=1200, height=1200)
dpg.setup_dearpygui()

In [7]:
with dpg.font_registry():
    with dpg.font(f'C:\\Windows\\Fonts\\calibri.ttf', 14, default_font=True, id="Default font"):
        dpg.add_font_range_hint(dpg.mvFontRangeHint_Cyrillic)
dpg.bind_font("Default font")

In [8]:
loadedImages = []
imageScale = 1
for imagePath, guid in imagePaths:
    width, height, channels, data = dpg.load_image(imagePath)
    imageScale = height / width
    with dpg.texture_registry(show=False):
        dpg.add_static_texture(width=width, height=height, default_value=data, tag=guid)

w = 80
h = w * imageScale

In [9]:
def listSearch_callback(sender, app_data, user_data):
    print(f"sender is: {sender}")
    print(f"app_data is: {app_data}")
    print(f"user_data is: {user_data}")
    dpg.delete_item("table_items", dpg.get_item_children("table_items"))

In [10]:
def checkbox_callback(sender, app_data, user_data):
    print(f"sender is: {sender}")
    print(f"app_data is: {app_data}")
    print(f"user_data is: {user_data}")

In [11]:
selected_item = None

In [12]:
def select_callback(sender, app_data, user_data):
    global selected_item
    print(f"sender is: {sender}")
    print(f"app_data is: {app_data}")
    print(f"user_data is: {user_data}")

    if selected_item is not None and selected_item != sender:
        dpg.configure_item(selected_item, default_value=False)

    #dpg.configure_item(sender, selected=False)
    selected_item = sender

In [13]:
def on_main_window_resize(sender, data):
    window_height = dpg.get_viewport_height() * 0.96
    dpg.set_item_height(item="pane_filters", height=window_height)  # пример изменения ширины окна
    dpg.set_item_height(item="pane_list", height=window_height)  # пример изменения ширины окна

In [14]:
with dpg.window(tag="pane_filters", label="Filters", width=200, height=1100, no_collapse=True, no_move=True, no_close=True, pos=[4, 4],no_resize=True):
    with dpg.group(horizontal=False):
        dpg.add_radio_button(label="Test",items=["Anime", "Hentai", "Music", "Manga", "Dorama"], default_value='Anime', callback=checkbox_callback)

        dpg.add_separator(label="SectionToGenreSeparator")

        with dpg.table(label="GenreCategories", header_row=True):
                dpg.add_table_column(label="",width=20,width_fixed=True)
                dpg.add_table_column(label="Category",width=w-20,width_fixed=True)
                with dpg.table_row():
                    dpg.add_checkbox(callback=checkbox_callback, user_data="Data1")
                    dpg.add_text("1")

In [15]:
with dpg.window(tag="pane_list", label="List", height=1100, pos=[210, 4], no_collapse=True, no_move=True, no_close=True,no_resize=True):
    with dpg.group(horizontal=False):
        dpg.add_input_text(callback=listSearch_callback)

        with dpg.table(label="table_items", header_row=True):

            # use add_table_column to add columns to the table,
            # table columns use child slot 0
            dpg.add_table_column(label="Poster",width=w,width_fixed=True)
            dpg.add_table_column(label="Name")
            dpg.add_table_column(label="", width=3)

            # add_table_next_column will jump to the next row
            # once it reaches the end of the columns
            # table next column use slot 1
            for item in items:
                with dpg.table_row():
                    imgItem = f"image:{item.guid}"
                    dpg.add_image(item.guid, width=w, height=h)
                    textItem = f"text:{item.guid}"
                    dpg.add_text(str(item.name), wrap=150, tag=textItem)
                    dpg.add_selectable(callback=select_callback, 
                                       span_columns=True, 
                                       height=max(h, dpg.get_item_height(textItem)))
                    
        with dpg.group(horizontal=True):
            dpg.add_button(label="1")
            dpg.add_button(label="2")
            dpg.add_button(label="3")

In [16]:
dpg.set_viewport_resize_callback(on_main_window_resize)

In [17]:
dpg.show_viewport()
dpg.start_dearpygui()
dpg.destroy_context()

sender is: 144
app_data is: True
user_data is: None
sender is: 148
app_data is: True
user_data is: None
sender is: 152
app_data is: True
user_data is: None
sender is: 160
app_data is: True
user_data is: None
sender is: 164
app_data is: True
user_data is: None
sender is: 168
app_data is: True
user_data is: None
sender is: 152
app_data is: True
user_data is: None
sender is: 144
app_data is: True
user_data is: None
sender is: 160
app_data is: True
user_data is: None


import dearpygui.dearpygui as dpg

dpg.create_context()

with dpg.window(label="Main Window"):
    with dpg.group(horizontal=True):
        dpg.add_button(label="Add Category")
    
    dpg.add_listbox(label="Categories", width=200)

    with dpg.table(header_row=True, row_background=True, height=400,
                   borders_innerH=True, borders_innerV=True):
        dpg.add_table_column(label="Image")
        dpg.add_table_column(label="Description")
    
    
dpg.create_viewport(title='Custom Title', width=800, height=600)
dpg.setup_dearpygui()
dpg.show_viewport()
dpg.start_dearpygui()
dpg.destroy_context()

import dearpygui.dearpygui as dpg

dpg.create_context()

with dpg.theme() as table_theme:
    with dpg.theme_component(dpg.mvTable):
        # dpg.add_theme_color(dpg.mvThemeCol_HeaderHovered, (255, 0, 0, 100), category=dpg.mvThemeCat_Core)
        dpg.add_theme_color(dpg.mvThemeCol_HeaderActive, (0, 0, 0, 0), category=dpg.mvThemeCat_Core)
        dpg.add_theme_color(dpg.mvThemeCol_Header, (0, 0, 0, 0), category=dpg.mvThemeCat_Core)

def clb_selectable(sender, app_data, user_data):
    print(f"Row {user_data}")

with dpg.window(tag="Selectable Tables"):
    with dpg.table(tag="SelectRows", header_row=True) as selectablerows:
        dpg.add_table_column(label="First")
        dpg.add_table_column(label="Second")
        dpg.add_table_column(label="Third")

        for i in range(15):
            with dpg.table_row():
                for j in range(3):
                    dpg.add_selectable(label=f"Row{i} Column{j}", span_columns=True, callback=clb_selectable, user_data=i)
    dpg.bind_item_theme(selectablerows, table_theme)

    with dpg.table(tag="SelectCells", header_row=True) as selectablecells:
        dpg.add_table_column(label="First")
        dpg.add_table_column(label="Second")
        dpg.add_table_column(label="Third")

        for i in range(15):
            with dpg.table_row():
                for j in range(3):
                    dpg.add_selectable(label=f"Row{i} Column{j}", callback=clb_selectable, user_data=(i,j))
    dpg.bind_item_theme(selectablecells, table_theme)

dpg.create_viewport(width=800, height=600)
dpg.setup_dearpygui()
dpg.show_viewport()
dpg.start_dearpygui()
dpg.destroy_context()